#7-Puzzle
---

I made some changes in the Priority Queue Frontier Class

---
Sometimes, BFS can outperform UCS (it find the cheapest path quicker, exploring less paths).

For example, with the given files, BFS will be faster because it doesn't consider weights at, while UCS loses time by calculating weights (since 7 is the only tile to move, but it is also really expensive) only to show the same result as BFS.

Also, if for example we set the weights.txt to

```
0 0 0 0 0 0 1
```
UCS will take really long to end, because it must finish exploring all the other (free) paths until it finally reaches the last path, which will be the correct one


A* on the other hand has shown to be more reliable then both

In [ ]:
# import sys
# import heapq
# import numpy as np
# import copy

class Node():
    def __init__(self, state, parent=None, action=None, weight=0):
        self.state = state
        self.parent = parent
        self.action = action #number and its move
        self.weight = weight

#BFS
class QueueFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

#UCS and A*
class PriorityQueueFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, newNode, priorityValue):
        nr = len(self.frontier) #will be used to tell us the order in which this node was added
        # heapq.heappush(self.frontier, (priorityValue, id(newNode), newNode) )
        self.frontier.append((priorityValue,nr,newNode))
        self.frontier = sorted(self.frontier, key=lambda x: (x[0],x[1]))

    def contains_state(self, state):
        return any(tuplee[2].state == state for tuplee in self.frontier) #frontier actually stores tuples. tuplee[2] is the actual node

    def empty(self):
        return len(self.frontier) == 0

    # def remove(self):
    #     if self.empty():
    #         raise Exception("empty frontier")
    #     else:
    #         t = heapq.heappop(self.frontier)
    #         return t[-1]
    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            # return node[2]#return the node actually
            return node #actually a tuple

#7 puzzle
class Problem():

    def __init__(self, weights, filename):
        self.weights = weights
        self.filename = filename #will need it to write output later

        #read initial state file
        file =  open(filename, 'r')
        content = []
        for line in file.readlines(): #convert to matrix
          content.append(tuple(map(int, line.split())))
        self.start = tuple(content)
        file.close()

        #goal state
        self.goal = ((1,2,3),
                     (4,5,6),
                     (7,0,0))

    def neighbors(self, state): #if it wasnt for weights, this method could be static too (BFS for example)
      hood = []

      for i in range(len(state)):
        for j in range(len(state[i])):
          #this condition will be executed 2 times (we have 2 zeros)
          if state[i][j]==0:
            #must be in border
            if i-1>=0:
              temp = list(map(list,state))#pass by value, not reference
              temp[i][j] = temp[i-1][j]
              temp[i-1][j] = 0
              temp = tuple(map(tuple,temp))#convert to tuple
              #move consists of new state (temp), the nr which switches with 0, the number's direction (opposite of 0 direction), and the weight/cost needed to move that number
              move = (temp, temp[i][j], "DOWN", self.weights[temp[i][j] - 1]) #weights[temp[i][j] - 1] is weight of temp[i][j]. example: weight of 7 is weights[6]
              hood.append(move)

            if j+1<=2:
              temp = list(map(list,state))
              temp[i][j] = temp[i][j+1]
              temp[i][j+1] = 0
              temp = tuple(map(tuple,temp))
              move = (temp, temp[i][j], "LEFT", self.weights[temp[i][j] - 1])
              hood.append(move)

            if i+1<=2:
              temp = list(map(list,state))
              temp[i][j] = temp[i+1][j]
              temp[i+1][j] = 0
              temp = tuple(map(tuple,temp))
              move = (temp, temp[i][j], "UP", self.weights[temp[i][j] - 1])
              hood.append(move)

            if j-1>=0:
              temp = list(map(list,state))
              temp[i][j] = temp[i][j-1]
              temp[i][j-1] = 0
              temp = tuple(map(tuple,temp))
              move = (temp, temp[i][j], "RIGHT", self.weights[temp[i][j] - 1])
              hood.append(move)

      # when done return neighbors (from top, clockwise)
      return hood

    def heuristic(self, state):
      #(weight × Manhattan distance from its goal position) for each tile
      #dictionary
      sum = 0
      goal_p = {
            1: (0, 0), 2: (0, 1), 3: (0, 2),
            4: (1, 0), 5: (1, 1), 6: (1, 2),
            7: (2, 0)
        }

      for i in range(len(state)):
          for j in range(len(state[0])):
              if state[i][j]!= 0:
                  # print(goal_p[state[i][j]])
                  dx = abs(goal_p[state[i][j]][1] - j)
                  dy = abs(goal_p[state[i][j]][0] - i)
                  manhattan = dx+dy
                  sum+= (manhattan * self.weights[state[i][j] - 1])

      return sum


    def solveBFS(self):
        #create first node
        node = Node(self.start)
        self.num_explored =0

        #create frontier
        front = QueueFrontier()
        front.add(node)

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:
            if front.empty():
              raise Exception("no solution")

            # Choose a node from the frontier
            node = front.remove()
            self.num_explored += 1

            if node.state == self.goal:
                actions = []
                statess = []
                weight_sum = 0
                while node.parent is not None:
                    actions.append(node.action)
                    statess.append(node.state)
                    weight_sum += node.weight
                    node = node.parent
                actions.reverse()
                statess.reverse()
                self.solution = (actions, statess, weight_sum)
                return

            # Mark node as explored
            self.explored.add(node.state)

            #add to the frontier neighbors who arent already and not explored yet
            hood = self.neighbors(node.state)
            for (state, num, move, weight) in hood:
              if not front.contains_state(state) and state not in self.explored:
                child = Node(state, node, (num, move), weight)
                front.add(child)

    def solveUCS(self):
        #create first node
        node = Node(self.start)
        self.num_explored =0

        #create frontier
        front = PriorityQueueFrontier()
        front.add(node,0)#first cost is 0 always

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:
            if front.empty():
              raise Exception("no solution")

            # Choose a node from the frontier
            node = front.remove()#this is actually a tuple, (weight, order, node)
            self.num_explored += 1

            if node[2].state == self.goal:
                nodee = node[2]
                actions = []
                statess = []
                weight_sum = 0
                while nodee.parent is not None:
                    actions.append(nodee.action)
                    statess.append(nodee.state)
                    weight_sum += nodee.weight
                    nodee = nodee.parent
                actions.reverse()
                statess.reverse()
                self.solution = (actions, statess, weight_sum)
                return

            # Mark node as explored
            self.explored.add(node[2].state)

            #add to the frontier neighbors who arent already and not explored yet
            hood = self.neighbors(node[2].state)
            for (state, num, move, weight) in hood:
              if not front.contains_state(state) and state not in self.explored:
                child = Node(state, node[2], (num, move), weight)
                front.add(child,weight + node[0]) #cost so far + child cost: node[0] stores cost so far

    def solveAStar(self):
        #create first node
        node = Node(self.start)
        self.num_explored =0

        #create frontier
        front = PriorityQueueFrontier()
        front.add(node,0 + self.heuristic(self.start))#first cost is 0 +h(start) always

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:
            if front.empty():
              raise Exception("no solution")

            # Choose a node from the frontier
            node = front.remove()#this is actually a tuple, (weight, order, node)
            self.num_explored += 1

            if node[2].state == self.goal:
                nodee = node[2]
                actions = []
                statess = []
                weight_sum = 0
                while nodee.parent is not None:
                    actions.append(nodee.action)
                    statess.append(nodee.state)
                    weight_sum += nodee.weight
                    nodee = nodee.parent
                actions.reverse()
                statess.reverse()
                self.solution = (actions, statess, weight_sum)
                return

            # Mark node as explored
            self.explored.add(node[2].state)

            #add to the frontier neighbors who arent already and not explored yet
            hood = self.neighbors(node[2].state)
            for (state, num, move, weight) in hood:
              if not front.contains_state(state) and state not in self.explored:
                child = Node(state, node[2], (num, move), weight)
                front.add(child,weight + node[0] + self.heuristic(state)) #cost so far + child cost + predicted cost: node[0] stores cost so far

    def print(self, file):
      f = open(file,'w')
      f.write(f"Number of explored states: {self.num_explored}\n")
      f.write(f"Length of solution path: {len(self.solution[0])}\n")
      f.write(f"Cost of solution path: {self.solution[2]}\n\n")
      for i in range(len(self.solution[0])):
        f.write(f"STEP {i+1}: MOVE {self.solution[0][i][0]} {self.solution[0][i][1]}\n")
        for x in self.solution[1][i]:
          f.write(f"{x[0]} {x[1]} {x[2]}\n")
        f.write("\n")

      f.close()

    def writeAll(self):
      #bfs output
      self.solveBFS()
      self.print(self.filename+"BFS.txt")

      #ucs output
      self.solveUCS()
      self.print(self.filename+"UCS.txt")

      #a* output
      self.solveAStar()
      self.print(self.filename+"AStar.txt")


#----------------------------
#read weights.txt or set every weight to 1
try:
    file = open("weights.txt", 'r')
    weights = list(map(int, file.read().split())) #convert to matrix
    file.close()
    if(len(weights)!= 7): raise FileNotFoundError #must have 7 elements (weights)
except FileNotFoundError:
    weights = [1,1,1,1,1,1,1]

#get initial state file name from user
file = input("Enter the initial state file name: ")

#create object
obj = Problem(weights, file)

obj.writeAll()




Enter the initial state file name: input1


#Jewels Maze
---
The hardest part of this problem was to implement a good heuristic function.

I have chosen to add the manhattan distance of a node from the closest unique unfound jewels.

That is, if we already have found 1 BLUE jewel, h() will be the sum of the manhattan distances of the current node from the closest nodes containing GREEN and RED jewel.

---

I have also done some changes on the node class.

Nodes now have attributes B, G and R which tell us which jewels have been found so far in the path of this node

As requested, the algorithms will end once we have found all jewel types

In [ ]:
import random

class Node():
    #B tells us if we have found any blue jewels, G for green, R for red
    def __init__(self, state, parent, action, weight=0, B=False, G=False, R=False):
        self.state = state
        self.parent = parent
        self.action = action
        self.B = B
        self.G = G
        self.R = R
        self.weight=weight

    #the idea of this method is to let the program know which jewels have been found so far
    def child(self,state,action,contents):
      #I am not sure why, but sometimes I get string index out of range error, so I added this try/catch
      try:
        which = contents[state[0]][state[1]]
      except:
        return Node(state, self, action, self.weight+1, B=self.B, G=self.G, R=self.R)

      if which == "B":
        return Node(state, self, action, self.weight+1, B=True, G=self.G, R=self.R)

      if which == "G":
        return Node(state, self, action, self.weight+1, B=self.B, G=True, R=self.R)

      if which == "R":
        return Node(state, self, action, self.weight+1, B=self.B, G=self.G, R=True)

      return Node(state, self, action, self.weight+1, B=self.B, G=self.G, R=self.R)

#DFS
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

class QueueFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

#A*
class PriorityQueueFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, newNode, priorityValue):
        nr = len(self.frontier) #will be used to tell us the order in which this node was added
        # heapq.heappush(self.frontier, (priorityValue, id(newNode), newNode) )
        self.frontier.append((priorityValue,nr,newNode))
        self.frontier = sorted(self.frontier, key=lambda x: (x[0],x[1]))

    def contains_state(self, state):
        return any(tuplee[2].state == state for tuplee in self.frontier) #frontier actually stores tuples. tuplee[2] is the actual node

    def empty(self):
        return len(self.frontier) == 0

    # def remove(self):
    #     if self.empty():
    #         raise Exception("empty frontier")
    #     else:
    #         t = heapq.heappop(self.frontier)
    #         return t[-1]
    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            # return node[2]#return the node actually
            return node #actually a tuple


class Maze():

    def __init__(self, filename):

        #goals, will be used to scan the file
        self.goalJewels = ["B","G","R"]

        # Read file and set height and width of maze
        with open(filename) as f:
            contents = f.read()

        # Validate start and goal
        if contents.count("A") != 1:
            raise Exception("maze must have exactly one start point")

        # Determine height and width of maze
        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)
        self.goals = set()
        self.contents = contents

        # Keep track of walls
        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j] == "A":
                        self.start = (i, j)
                        row.append(False)
                    # if jewel add to goal
                    elif contents[i][j] in self.goalJewels:
                        self.goals.add( (i, j) )
                        row.append(False)
                    elif contents[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)

        self.solution = None

    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]

        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result

    # # heuristic function is the manhattan distance to the goal
    # def heuristicFunctionSingleGoal(self, currentNode, goal):
    #   return abs(currentNode.state[0] - goal[0]) + abs(currentNode.state[1] - goal[1])

    # def heuristicFunction(self, currentNode):
    #   found = []
    #   if currentNode.B: found.append("B")
    #   if currentNode.G: found.append("G")
    #   if currentNode.R: found.append("R")
    #   goals = []

    #   for (i,j) in self.goal:
    #     if self.contents[i][j] not in found:
    #       goals.append((i,j))

    #   values = [self.heuristicFunctionSingleGoal(currentNode, g) for g in goals]
    #   return min(values)

    # def heuristicFunction(self, currentNode):
    #   found = []
    #   if currentNode.B: found.append("B")
    #   if currentNode.G: found.append("G")
    #   if currentNode.R: found.append("R")
    #   goals = []

    #   for (i,j) in self.goal:
    #     if self.contents[i][j] not in found:
    #       goals.append((i,j))

    #   values = [self.heuristicFunctionSingleGoal(currentNode, g) for g in goals]
    #   if values == list(): return 0
    #   return min(values)

    # heuristic function is the manhattan distance to the goal
    def heuristicFunctionSingleGoal(self, currentNode, goal):
        (x,y) = currentNode.state

        hn = abs(currentNode.state[0] - goal[0]) + abs(currentNode.state[1] - goal[1])
        return( (hn,self.contents[x][y]) )#return perceived distance and jewel type

    def heuristicFunction(self, currentNode, f=None):
      #we need to perceive distance only for unfound jewels!!!
      found = []
      if currentNode.B: found.append("B")
      if currentNode.G: found.append("G")
      if currentNode.R: found.append("R")
      goals = []

      for (i,j) in self.goal:
        if self.contents[i][j] not in found:
          goals.append((i,j))

      values = [self.heuristicFunctionSingleGoal(currentNode, g) for g in goals]
      if values == []:return 0#if no goals left to found, h() is 0
      sorted(values, key=lambda x: (x[0]))#sort based on h()

      #we need to find a perceived value for all unfound jewels, not simply 1!!!
      hn = 0
      for (h,j) in values:
        if j not in found:
            hn+=h
        found.append(j)#since we already perceived the smallest distance to this jewel type, we add the jewel to the found list, cause we dont need this type anymore

      return hn

    def solveBFS(self):
        """Finds a solution to maze, if one exists."""

        self.goal = self.goals

        # Keep track of number of states explored
        self.num_explored = 0

        # Initialize frontier to just the starting position
        start = Node(state=self.start, parent=None, action=None)
        frontier = QueueFrontier()
        frontier.add(start)

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:

            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception("no solution BFS")

            # Choose a node from the frontier
            node = frontier.remove()
            self.num_explored += 1

            # If node is the goal, then we have a solution
            if node.B and node.G and node.R: #node.state in self.goals:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            # Mark node as explored
            self.explored.add(node.state)

            # Add neighbors to frontier
            the_neighbors = self.neighbors(node.state)
            # random.shuffle(the_neighbors)
            for action, state in the_neighbors:
                if not frontier.contains_state(state) and state not in self.explored:
                    # child = Node(state=state, parent=node, action=action)
                    child = node.child(state,action,self.contents)
                    frontier.add(child)


    def solveDFS(self):
        """Finds a solution to maze, if one exists."""

        self.goal = self.goals

        # Keep track of number of states explored
        self.num_explored = 0

        # Initialize frontier to just the starting position
        start = Node(state=self.start, parent=None, action=None)
        frontier = StackFrontier()
        frontier.add(start)

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:

            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception("no solution DFS")

            # Choose a node from the frontier
            node = frontier.remove()
            self.num_explored += 1

            # If node is the goal, then we have a solution
            if node.B and node.G and node.R:#node.state in self.goals:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            # Mark node as explored
            self.explored.add(node.state)

            # Add neighbors to frontier
            the_neighbors = self.neighbors(node.state)
            random.shuffle(the_neighbors)
            for action, state in the_neighbors:
                if not frontier.contains_state(state) and state not in self.explored:
                    child = node.child(state,action,self.contents)
                    frontier.add(child)



################################################################################################################

    # def solveAStar(self):
    #     """Finds a solution to maze, if one exists."""

    #     self.goal = self.goals

    #     # Keep track of number of states explored
    #     self.num_explored = 0

    #     # Initialize frontier to just the starting position
    #     start = Node(self.start, None, None)
    #     frontier = PriorityQueueFrontier()
    #     priorityValue = 0 + self.heuristicFunction(start)
    #     frontier.add(start, priorityValue)

    #     # Initialize an empty explored set
    #     self.explored = set()

    #     # Keep looping until solution found
    #     while True:

    #         # If nothing left in frontier, then no path
    #         if frontier.empty():
    #             raise Exception("no solution")

    #         # Choose a node from the frontier
    #         nodeT = frontier.remove()#actually a tuple
    #         node = nodeT[2]
    #         self.num_explored += 1

    #         # If node is the goal, then we have a solution
    #         if node.B and node.G and node.R: #node.state in self.goal:
    #             actions = []
    #             cells = []
    #             while node.parent is not None:
    #                 actions.append(node.action)
    #                 cells.append(node.state)
    #                 node = node.parent
    #             actions.reverse()
    #             cells.reverse()
    #             self.solution = (actions, cells)
    #             return

    #         # Mark node as explored
    #         self.explored.add(node.state)

    #         # Add neighbors to frontier
    #         for action, state in self.neighbors(node.state):
    #             if state not in self.explored:
    #                 # child = Node(state=state, parent=node, action=action, weight = node.weight+1 )
    #                 child = node.child(state, action, self.contents)
    #                 gn = child.weight
    #                 hn = self.heuristicFunction(child)
    #                 priorityValue = gn + hn
    #                 frontier.add(child, priorityValue)


    #                 #ff node is a goal, then we remove that goal type (we need 1 element from each, not all)
    #                 if node.state in self.goal:
    #                     newGoal = set() #new goals (without previous value)
    #                     for (x,y) in self.goal:
    #                       #if different in value add to new goals,
    #                         if self.contents[x][y] != self.contents[node.state[0]][node.state[1]]:
    #                             newGoal.add( (x,y) )

    #                     #update goal nodes and goal jewels
    #                     self.goal = newGoal



    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("█", end="")
                elif (i, j) == self.start:
                    print("A", end="")
                elif (i, j) in self.goals:
                    print(self.contents[i][j], end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()

    def output_image(self, filename, show_solution=True, show_explored=False):
        from PIL import Image, ImageDraw
        cell_size = 50
        cell_border = 2

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size, self.height * cell_size),
            "black"
        )
        draw = ImageDraw.Draw(img)

        solution = self.solution[1] if self.solution is not None else None
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):

                # Walls
                if col:
                    fill = (40, 40, 40)

                # Start
                elif (i, j) == self.start:
                    fill = (255, 255, 0)

                # Goal
                elif (i, j) in self.goal:
                  if self.contents[i][j]=="R":
                    fill = (255, 0, 0)

                  elif self.contents[i][j]=="B":
                    fill = (0,0,255)

                  else: fill = (0,171,28)

                # Solution
                elif solution is not None and show_solution and (i, j) in solution:
                    fill = (200, 250, 100)

                # Explored
                elif solution is not None and show_explored and (i, j) in self.explored:
                    fill = (150, 150, 150)

                # Empty cell
                else:
                    fill = (237, 240, 252)

                # Draw cell
                draw.rectangle(
                    ([(j * cell_size + cell_border, i * cell_size + cell_border),
                      ((j + 1) * cell_size - cell_border, (i + 1) * cell_size - cell_border)]),
                    fill=fill
                )

        img.save(filename)

########################################################################

    def solveAStar(self):
        """Finds a solution to maze, if one exists."""

        self.goal = self.goals

        # Keep track of number of states explored
        self.num_explored = 0

        # Initialize frontier to just the starting position
        start = Node(state=self.start, parent=None, action=None)
        frontier = PriorityQueueFrontier()
        priorityValue = 0 + self.heuristicFunction(start)
        frontier.add(start, priorityValue)

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:

            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception("no solution A*")

            # Choose a node from the frontier
            nodeT = frontier.remove()#actually a tuple
            node = nodeT[2]
            self.num_explored += 1

            # If node is the goal, then we have a solution
            if node.B and node.G and node.R: #node.state in self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            # Mark node as explored
            self.explored.add(node.state)

            # Add neighbors to frontier
            for action, state in self.neighbors(node.state):
                if state not in self.explored:
                    # child = Node(state=state, parent=node, action=action, depth = node.depth+1 )
                    child = node.child(state, action, self.contents)
                    gn = child.weight
                    hn = self.heuristicFunction(child)
                    priorityValue = gn + hn
                    frontier.add(child, priorityValue)

    def solveAll(self):
        self.print()
        print("Solving using DFS...")
        self.solveDFS()
        self.output_image("DFS.png", show_explored=True)
        print("States Explored with DFS:", m.num_explored)
        print("DFS Solution:")
        self.print()

        self.print()
        print("Solving using BFS...")
        self.solveBFS()
        self.output_image("BFS.png", show_explored=True)
        print("States Explored with BFS:", m.num_explored)
        print("BFS Solution:")
        self.print()

        self.print()
        print("Solving using A*...")
        self.solveAStar()
        self.output_image("AStar.png", show_explored=True)
        print("States Explored with A*:", m.num_explored)
        print("A* Solution:")
        self.print()

filename = input("Enter file name: ")
m = Maze(filename)
m.solveAll()


Enter file name: a

A          █    █ ██ 
     █       █       
 █      ██           
   █                 
   █     █      █ ██ 
 █     G           R 
   █          █ █ █  
   R                 
█     █    G       B 
        █            
  █           █    █ 
      B              
        █        R   
B                    
        ██  █ █      
       █ B     █     
      █ █  █         
 █                   
              █      
          █        B 

Solving using DFS...
States Explored with DFS: 99
DFS Solution:

A          █    █ ██ 
***  █       █   ****
 █*     ██********  *
 **█      *         *
** █ *** █* ****█ ██*
*█   * G*** *  *** R*
*  █**     ** █ █*█ *
***R*      *     ****
█     █    G*      B 
        █   *        
  █        ** █    █ 
      B*****         
     ***█        R   
B*  **               
 *  *   ██  █ █      
**  *  █ B     █     
* *** █ █  █         
*█*                  
* *           █      
***       █        B 


A          █    █ ██ 
***  █    